In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import yfinance as yf
from caviar import CaviarModel
from var_tests import *
import matplotlib.pyplot as plt

In [ ]:
df = yf.download("AAPL JPM SPY XOM COKE", start="2001-01-01")

In [ ]:
df_log_ret = df.Close.apply(np.log).diff().dropna() * 100
df_log_ret

## Emperical Analysis - Implementation (SPY)

In [ ]:
quantiles = [0.01, 0.05]
models = ['adaptive', 'asymmetric', 'symmetric', 'igarch']
tickers = ['SPY'] # df_log_ret.columns

In [ ]:
out_samples.shape

In [ ]:
stats_df = []
beta_stats_df = []

save_csv_path = '../result/implementation'
save_plot_path = '../result/implementation/plots'

for ticker in tickers:
    in_samples = df_log_ret.loc[:'2015', ticker]
    out_samples = df_log_ret.loc['2016':, ticker]
    
    for q in quantiles:
        for model in models:
            caviar_model = CaviarModel(quantile=q, model=model, method='RQ')
            caviar_model.fit(in_samples)
            
            # save plots
            fig1 = caviar_model.plot_caviar(in_samples, 'in')
            plt.savefig(f'{save_plot_path}/{model}_{int(q*100)}%_plt_caviar_in.jpg')
            fig2 = caviar_model.plot_caviar(out_samples, 'out')
            plt.savefig(f'{save_plot_path}/{model}_{int(q*100)}%_plt_caviar_out.jpg')
            fig3 = caviar_model.plot_news_impact_curve()
            plt.savefig(f'{save_plot_path}/{model}_{int(q*100)}%_plt_news_impact_curve.jpg')
                        
            # as the last prediction is the VaR forecast
            in_VaR = caviar_model.predict(in_samples, caviar_model.VaR0_in)[:-1]
            out_VaR = caviar_model.predict(out_samples, caviar_model.VaR0_out)[:-1]

            # statistics
            stat = {
                'quantile': q,
                'model': model,
                'method': 'RQ',
                # 'trial': i,
                
                # insamples
                'loss': caviar_model.training_loss,
                'hit_rate_in': hit_rate(in_samples, in_VaR),
                'binom_in': binomial_test(in_samples, in_VaR, q),
                'traffic_in': traffic_light_test(in_samples, in_VaR, q)[0],
                'kupiec_in': kupiec_pof_test(in_samples, in_VaR, q),
                'independent_in': christoffersen_test(in_samples, in_VaR),
                'dq_in': caviar_model.dq_test(in_samples, 'in'),
                # outsamples
                'hit_rate_out': hit_rate(out_samples, out_VaR),
                'binom_out': binomial_test(out_samples, out_VaR, q),
                'traffic_out': traffic_light_test(out_samples, out_VaR, q)[0],
                'kupiec_out': kupiec_pof_test(out_samples, out_VaR, q),
                'independent_out': christoffersen_test(out_samples, out_VaR),
                'dq_out': caviar_model.dq_test(out_samples, 'out'),                
            }

            beta_stat = caviar_model.beta_summary()
            beta_stat['quantile'] = q
            beta_stat['model'] = model
            beta_stat['method'] = 'RQ'
            # beta_stat['trial'] = i

            stats_df.append(stat)
            beta_stats_df.append(beta_stat)

            
# save data
stats_df = pd.DataFrame(stats_df)
beta_stats_df = pd.concat(beta_stats_df)
beta_stats_df.index.name = 'beta'

stats_df.T.to_csv(f'{save_csv_path}/experiment_stat_data_RQ_ppt.csv', header=None)
beta_stats_df.T.to_csv(f'{save_csv_path}/experiment_beta_stat_data_ppt.csv')